# Парсер сайта https://itab.pro/

Сайт содержит самый большой список проверенных российских БАДов с описанием и ценой

In [10]:
!pip install beautifulsoup4 requests

## Загружаем необходимые библиотеки

In [1]:
import os
import pandas as pd
import re
from lxml import html
import requests
from tqdm.notebook import tqdm
from time import sleep

tqdm.pandas()

## Определяем необходимые функции

In [2]:
# Функция обьединяет иерархический список текста описание в единый и удалеет таги из списка 
# и также удаляет некоторые символы html разметки
def preprocessing_text(text_list):
    new_text_list = []
    for item in text_list:
        if item[0] == 'p':
            new_text_list.append(re.sub(r"\xa0", ' ', "".join(sent for sent in item[1])) + "\n")
            
        elif item[0] == 'li': 
            new_text_list.append(re.sub(r"\xa0", ' ', "".join(sent for sent in item[1])) + "\n") 
            
        elif item[0] == 'tr':
            new_text_list.append(re.sub(r"\xa0", ' ', "".join(sent + " " for sent in item[1])).strip() + "\n") 
                
    return new_text_list      

In [3]:
# Функция проходит по ссылам на описание продукта и возвращает предобработанное описание
def get_desc_by_link(link):
    prices_clean = []
    price_list = []
    product_desc = []
    
    response = requests.get(link, headers=headers)
    sleep(1)
    itab_dom = html.fromstring(response.text)
    
    # Получение наименования и описания БАДа
    product_desc_html = itab_dom.xpath("//div[@class='html-area']")
    
    for node in product_desc_html:
        for item in node:
            if item.tag == 'p':
                product_desc.append([item.tag, item.xpath(".//text()")])
            elif item.tag == 'ul':
                for subitem in item:
                    product_desc.append([subitem.tag, subitem.xpath(".//text()")])
            elif item.tag == 'table':
                for subitem in item:
                    product_desc.append([subitem.tag, subitem.xpath(".//text()")])
    
    # Получение цены БАДа
    product_price_html = itab_dom.xpath("//div[@class='quantity-counter-area']")
    
    for price in product_price_html:
        prices = price.xpath(".//text()")
        for i in range(len(prices)):
            prices_clean.append(re.sub(r"\xa0₽", '', prices[i]))
        price_list = [int(price) for price in prices_clean if price.isdigit()]
    
    if len(price_list) > 1:
        return preprocessing_text(product_desc), price_list[1], price_list[0]
    elif len(price_list) == 1:
        return preprocessing_text(product_desc), price_list[0], 0
    else:
        return preprocessing_text(product_desc), 0, 0

In [4]:
def join_text_from_list(text_list):
    return "".join(sent for sent in text_list)

## Выполняем загрузку и парсинг данных с сайта itab.pro

### Загружаем наименование и список ссылок на описание всех продуктов

In [5]:
# Определим заголовок для выполнения запроса
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
}

url = "https://itab.pro/products/"
url_next_page = "https://itab.pro/products/?page="
url_head = "https://itab.pro"
URL_PAGES = 56

In [6]:
DS_FILE_PATH = "itab_bad_products.csv" 

In [7]:
# Получаем наименование всех БАДов, ссылку на описание
if not os.path.isfile(DS_FILE_PATH):
    response = requests.get(url, headers=headers)
    full_product_links = []
    full_product_names = []
    for i in tqdm(range(URL_PAGES)):
        if i == 0:
            response = requests.get(url, headers=headers)
        else:
            url_page = url_next_page + str(i)
            response = requests.get(url_page, headers=headers)
        
        itab_dom = html.fromstring(response.text)
    
        products_link = itab_dom.xpath("//div[@class='row']/div/div/div[@class='product-card-top']/div/a/@href")
        products_link = [url_head + link for link in products_link]
        products_name = itab_dom.xpath("//div[@class='row']/div/div/div[@class='product-card-top']/div/a[1]/text()")
        full_product_links.extend(products_link)
        full_product_names.extend(products_name)
        sleep(1)

  0%|          | 0/56 [00:00<?, ?it/s]

In [13]:
len(full_product_links), len(full_product_names)

(1680, 1680)

Создаем датасет БАДов 

In [14]:
products_df = pd.DataFrame(data=full_product_names, columns=['bad_name'])
products_df['link'] = full_product_links

In [15]:
products_df.head(3)

,bad_name,link
0,"SuperMins, Витамин D3, жидкость, 10 мл",https://itab.pro/products/supermins-vitamin-d3...
1,"BioExpert, Биоактивный магниевый комплекс (5 ф...",https://itab.pro/products/bioexpert-bioaktivny...
2,"Wolfsport+iTAB, Вкусный B-комплекс (вишня), шо...",https://itab.pro/products/wolfsportitab-vkusny...


Загружаем цены и описание для каждого БАДА

In [24]:
full_description_list = []
full_discount_price_list = []
full_regular_price_list = []

if not os.path.isfile(DS_FILE_PATH):
    for i in tqdm(range(products_df.shape[0])):
        # Получаем описание стандартную цену и цену с учетом скидки или акции по url БАДа
        desc, rp, dp = get_desc_by_link(products_df["link"][i])
        desc = join_text_from_list(desc)
        
        full_description_list.append(desc)
        full_discount_price_list.append(dp)
        full_regular_price_list.append(rp)

  0%|          | 0/1680 [00:00<?, ?it/s]

In [25]:
len(full_description_list), len(full_discount_price_list), len(full_regular_price_list)

(1680, 1680, 1680)

In [26]:
products_df['Description'] = full_description_list
products_df['Regular_Price'] = full_regular_price_list
products_df['Discounted_Price'] = full_discount_price_list

In [27]:
products_df.head(3)

,bad_name,link,Description,Regular_Price,Discounted_Price
0,"SuperMins, Витамин D3, жидкость, 10 мл",https://itab.pro/products/supermins-vitamin-d3...,Жидкий витамин D3 (2000 МЕ) от SuperMins - это...,670,463
1,"BioExpert, Биоактивный магниевый комплекс (5 ф...",https://itab.pro/products/bioexpert-bioaktivny...,Биоактивный магниевый комплекс (5 форм) от Bio...,2271,0
2,"Wolfsport+iTAB, Вкусный B-комплекс (вишня), шо...",https://itab.pro/products/wolfsportitab-vkusny...,Смотреть видео\nВ-комплекс от Wolfsport и iTAB...,3515,0


### Сохраняем датасет в csv файл

In [28]:
products_df.to_csv(DS_FILE_PATH, sep="|")